In [2]:
import itertools
import numpy as np

somelists = [
#    [1, 2],
#    [2, 3],
#    [1, 3]
    ['0', '0~'],
    ['1', '1~'],
    ['0', '0~']
]

lists = [
         [1, 2],
         [1, 3],
         [2, 3]
]

print(*itertools.product(*somelists))
print(*itertools.product(*lists))

#for element in itertools.product(*somelists):
#    print(element)

('0', '1', '0') ('0', '1', '0~') ('0', '1~', '0') ('0', '1~', '0~') ('0~', '1', '0') ('0~', '1', '0~') ('0~', '1~', '0') ('0~', '1~', '0~')
(1, 1, 2) (1, 1, 3) (1, 3, 2) (1, 3, 3) (2, 1, 2) (2, 1, 3) (2, 3, 2) (2, 3, 3)


In [3]:
from sympy.logic.boolalg import to_cnf
from sympy.abc import A, B, C, D, E, F, G, H, J


# to_cnf((A & B) | (C & D) | (E & F) | (G & H))
to_cnf((A & B) | (C & D) | (E & F))
# to_cnf((~A & ~B) | (~C & ~D) | (~E & ~F))
# to_cnf((A & B & C) | (D & E & F))

(A | C | E) & (A | C | F) & (A | D | E) & (A | D | F) & (B | C | E) & (B | C | F) & (B | D | E) & (B | D | F)

# Packages

In [4]:
%%capture
!pip install python-sat

In [5]:
from pysat.examples.fm import FM
from pysat.formula import WCNF
from itertools import combinations
import math

# Task

In [6]:
from pysat.formula import CNF
from pysat.formula import WCNF

In [7]:
n_neurons = 3
n_train = 3
n_features = n_neurons

assert(n_neurons % 2 == 1)  # odd number of neurons is allowed

def check_if_2nd_feature_true(X):
    return [x_i[1] is True for x_i in X]

def prepare_dataset(n_train_, n_features_, func_):
    X = [
         [True, False, False],
         [False, True, False],
         [True, True, False]
    ]
    Y = func_(X)
    return X, Y

X, Y = prepare_dataset(
    n_train_ = n_train, 
    n_features_ = n_features,
    func_ = check_if_2nd_feature_true
)

In [8]:
print(X)
print(Y)

[[True, False, False], [False, True, False], [True, True, False]]
[False, True, True]


In [9]:
wcnf = WCNF()

wcnf.append([-1, 2]) 
wcnf.append([-2, 3])
wcnf.append([-3, True])
wcnf.append([4, True])

print(wcnf.soft)
print(wcnf.hard)
print(wcnf.wght)

fm = FM(wcnf,verbose=10)
fm.compute()

[]
[[-1, 2], [-2, 3], [-3, True], [4, True]]
[]


True

In [10]:
# def encode_features(X):
#     n = len(X)
#     m = len(X[0])
#     print(f'Shape of X: n = {n}, m = {m}')
#     prop_features = [[i * n + j + 1 for j in range(m)] for i in range(n)]
#     return np.array(prop_features)


# Encoding weights
prop_weights_indices = [i for i in range(n_neurons)]

In [11]:
prop_weights_indices

[0, 1, 2]

In [12]:
group_size = int(math.ceil(n_neurons / 2))
assert group_size == math.floor(n_neurons / 2 + 1)

In [13]:
combos = list(combinations(prop_weights_indices, group_size))
print(combos[:3])

[(0, 1), (0, 2), (1, 2)]


In [14]:
cartesian_prod = list(itertools.product(*combos))
print(cartesian_prod)

[(0, 0, 1), (0, 0, 2), (0, 2, 1), (0, 2, 2), (1, 0, 1), (1, 0, 2), (1, 2, 1), (1, 2, 2)]


In [52]:
clauses_cartesian_products = []
for c in cartesian_prod:
    print(f'Combo: {c}')
    l = [[f'{symb}', f'~{symb}'] for symb in c]
    l_cart_prod = list(itertools.product(*l))
    clauses_cartesian_products += l_cart_prod
    print(l)
    print(l_cart_prod)

Combo: (0, 0, 1)
[['0', '~0'], ['0', '~0'], ['1', '~1']]
[('0', '0', '1'), ('0', '0', '~1'), ('0', '~0', '1'), ('0', '~0', '~1'), ('~0', '0', '1'), ('~0', '0', '~1'), ('~0', '~0', '1'), ('~0', '~0', '~1')]
Combo: (0, 0, 2)
[['0', '~0'], ['0', '~0'], ['2', '~2']]
[('0', '0', '2'), ('0', '0', '~2'), ('0', '~0', '2'), ('0', '~0', '~2'), ('~0', '0', '2'), ('~0', '0', '~2'), ('~0', '~0', '2'), ('~0', '~0', '~2')]
Combo: (0, 2, 1)
[['0', '~0'], ['2', '~2'], ['1', '~1']]
[('0', '2', '1'), ('0', '2', '~1'), ('0', '~2', '1'), ('0', '~2', '~1'), ('~0', '2', '1'), ('~0', '2', '~1'), ('~0', '~2', '1'), ('~0', '~2', '~1')]
Combo: (0, 2, 2)
[['0', '~0'], ['2', '~2'], ['2', '~2']]
[('0', '2', '2'), ('0', '2', '~2'), ('0', '~2', '2'), ('0', '~2', '~2'), ('~0', '2', '2'), ('~0', '2', '~2'), ('~0', '~2', '2'), ('~0', '~2', '~2')]
Combo: (1, 0, 1)
[['1', '~1'], ['0', '~0'], ['1', '~1']]
[('1', '0', '1'), ('1', '0', '~1'), ('1', '~0', '1'), ('1', '~0', '~1'), ('~1', '0', '1'), ('~1', '0', '~1'), ('~1', '~

In [53]:
clauses_cartesian_products

[('0', '0', '1'),
 ('0', '0', '~1'),
 ('0', '~0', '1'),
 ('0', '~0', '~1'),
 ('~0', '0', '1'),
 ('~0', '0', '~1'),
 ('~0', '~0', '1'),
 ('~0', '~0', '~1'),
 ('0', '0', '2'),
 ('0', '0', '~2'),
 ('0', '~0', '2'),
 ('0', '~0', '~2'),
 ('~0', '0', '2'),
 ('~0', '0', '~2'),
 ('~0', '~0', '2'),
 ('~0', '~0', '~2'),
 ('0', '2', '1'),
 ('0', '2', '~1'),
 ('0', '~2', '1'),
 ('0', '~2', '~1'),
 ('~0', '2', '1'),
 ('~0', '2', '~1'),
 ('~0', '~2', '1'),
 ('~0', '~2', '~1'),
 ('0', '2', '2'),
 ('0', '2', '~2'),
 ('0', '~2', '2'),
 ('0', '~2', '~2'),
 ('~0', '2', '2'),
 ('~0', '2', '~2'),
 ('~0', '~2', '2'),
 ('~0', '~2', '~2'),
 ('1', '0', '1'),
 ('1', '0', '~1'),
 ('1', '~0', '1'),
 ('1', '~0', '~1'),
 ('~1', '0', '1'),
 ('~1', '0', '~1'),
 ('~1', '~0', '1'),
 ('~1', '~0', '~1'),
 ('1', '0', '2'),
 ('1', '0', '~2'),
 ('1', '~0', '2'),
 ('1', '~0', '~2'),
 ('~1', '0', '2'),
 ('~1', '0', '~2'),
 ('~1', '~0', '2'),
 ('~1', '~0', '~2'),
 ('1', '2', '1'),
 ('1', '2', '~1'),
 ('1', '~2', '1'),
 ('1', '

In [54]:
import numpy as np

print(len(clauses_cartesian_products))
len(np.unique(clauses_cartesian_products))
len(set(clauses_cartesian_products))

64


64

In [57]:
def encode_prop_features(X):
    n = len(X)
    m = len(X[0])
    prop_features = []
    for i in range(n):
        vals = [i * n + j for j in range(m)]
        prop_features.append(vals)
    

    for i in range(n):
        for j in range(m):
            print(f'X[{i}][{j}] is encoded as {prop_features[i][j]}')

    return prop_features

prop_features = encode_prop_features(X)
print(prop_features)

X[0][0] is encoded as 0
X[0][1] is encoded as 1
X[0][2] is encoded as 2
X[1][0] is encoded as 3
X[1][1] is encoded as 4
X[1][2] is encoded as 5
X[2][0] is encoded as 6
X[2][1] is encoded as 7
X[2][2] is encoded as 8
[[0, 1, 2], [3, 4, 5], [6, 7, 8]]


In [62]:
s = '~23'
s = s.replace('~', '')
print(int(s))

23


In [67]:
for i in range(len(X)):
    for c in clauses_cartesian_products:
        # ('0', '0', '1')
        # Here we optimize - we dont need to add identical clauses
        
        # print(set(c), c)
        clause = []
        unique_clause_symbols = set(c)  # ('0', '0', '1') --> ('0', '1')
        # '1' signifies ~w1 V x_i1
        # '~1' signifies w1 V ~x_i1
        for symbol in unique_clause_symbols:  # ('0', '~2', '1')
            if not '~' in symbol:
                # then (e.f. for '1') its not(w_1) V (x_i1)
                index = int(symbol)
                clause.append(-prop_weights_indices[index])
                clause.append(prop_features[i][index])
            elif '~' in symbol:
                # then (e.f. for '1') its (w_1) V not(x_i1)
                symbol = symbol.replace('~', '')
                index = int(symbol)

                clause.append(prop_weights_indices[index])
                clause.append(-prop_features[i][index])
        optimized_clause = list(set(clause))
        print(f'Our clause: {clause} -> {optimized_clause}')


Our clause: [-1, 1, 0, 0] -> [0, 1, -1]
Our clause: [1, -1, 0, 0] -> [0, 1, -1]
Our clause: [0, 0, -1, 1, 0, 0] -> [0, 1, -1]
Our clause: [0, 0, 1, -1, 0, 0] -> [0, 1, -1]
Our clause: [0, 0, -1, 1, 0, 0] -> [0, 1, -1]
Our clause: [0, 0, 1, -1, 0, 0] -> [0, 1, -1]
Our clause: [0, 0, -1, 1] -> [0, 1, -1]
Our clause: [0, 0, 1, -1] -> [0, 1, -1]
Our clause: [-2, 2, 0, 0] -> [0, 2, -2]
Our clause: [2, -2, 0, 0] -> [0, 2, -2]
Our clause: [0, 0, -2, 2, 0, 0] -> [0, 2, -2]
Our clause: [0, 0, 2, -2, 0, 0] -> [0, 2, -2]
Our clause: [0, 0, -2, 2, 0, 0] -> [0, 2, -2]
Our clause: [0, 0, 2, -2, 0, 0] -> [0, 2, -2]
Our clause: [0, 0, -2, 2] -> [0, 2, -2]
Our clause: [0, 0, 2, -2] -> [0, 2, -2]
Our clause: [-1, 1, -2, 2, 0, 0] -> [0, 1, 2, -2, -1]
Our clause: [1, -1, -2, 2, 0, 0] -> [0, 1, 2, -2, -1]
Our clause: [2, -2, -1, 1, 0, 0] -> [0, 1, 2, -1, -2]
Our clause: [2, -2, 1, -1, 0, 0] -> [0, 1, 2, -1, -2]
Our clause: [0, 0, -1, 1, -2, 2] -> [0, 1, 2, -2, -1]
Our clause: [0, 0, -2, 2, 1, -1] -> [0, 1,

In [29]:


for x, y in zip(X, Y):
    if y is True:
        print(f'y is True, add clauses')



('0', '0', '1')

In [ ]:
# Weighted CNF form
wcnf = WCNF()


for x, y in zip(X, Y):
    print(f'Encoding x = {x}, y = {y}')
    
    # At that step, we have combinations of weight indices
    # For example, for 3 weights
    # [(w0 == x_i0) & (w1 == x_i1)] V [(w0 == x_i0) & (w2 == x_i2)] V [(w1 == x_i1) & (w2 == x_i2)]

    # Each combination in cartesian_prod(w_combinations) - is a special clause:
    # For example, for n_weights == 3, after cartesian product, all combinations:
    # (001) (002) (021) (022) (101) (102) (121) (122)
    # Which means (for the first clause (001)):
    # (w_0 == x_i0) | (w_0 == x_i0) | (w_1 == x_i1)

    # However, we still did not expand the equivalence (==):
    # [(~w_0 V x_i0) & (w_0 V ~x_i0)] V [(~w_0 V x_i0) & (w_0 V ~x_i0)] V [(~w_1 V x_i1) & (w_1 V ~x_i1)]

    # And this also has to be transformed into CNF:
    # Do substitution:
    # [(0) & (~0)] V [(0) & (~0)] V [(1) & (~1)]
    # Where 0: ~w_0 V x_i0, ~0: w_0 V ~x_i0
    # We have to transfer (001) -> [[0, ~0], [0, ~0], [1, ~1]] and make a cartesian product of it

    

    

    # for w_combination in combinations(propositional_weights, group_size):
    #     # print(c, c[0], c[1])
    #     # print(propositional_weights.index(c[0]), propositional_weights.index(c[1]))
    #     print(f'weight combination = {w_combination}')
        
    #     for weight in w_combination:
    #         # index of weight --> the same as index of feature of current object x_i in the X dataset
    #         w_index = propositional_weights.index(weight)
    #         x_feature = x[w_index]
    #         print(f'w_{w_index}, x_j = {x_feature}')

    #         # Our clause is - weight == x_feature (equivalence)
    #         # which is (not(weight) V x_feature) and (not(x_feature) V weight)
            
    #         first_clause = [-weight, x_feature]
    #         second_clause = [weight, -x_feature]
    #         wcnf.append



NameError: ignored

In [ ]:
# No hard clauses are introduced.
# Soft clauses - have weight 1, if the prediction of the network is correct
# for a given instance (x_i, y_i) in a dataset

In [ ]:
# Encode this into clauses
for x_i, y_i in zip(X, Y):
    print(f'Encoding x_i = {x_i}, y_i = {y_i}')
    

Encoding x_i = [True, False, False], y_i = False
Encoding x_i = [False, True, False], y_i = True
Encoding x_i = [True, True, False], y_i = True
